
# Neural Network Model Development

This notebook demonstrates the development of a custom neural network using TensorFlow and Keras, focusing on good coding practices and clear documentation.

### Library Imports
All necessary libraries are imported here for better organization.


In [ ]:
!pip install keras

In [3]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Synthetic Sensor data

-------------------------

In [7]:
df = pd.read_csv('/content/synthetic_datasetV2.csv')
df.head(10)
#df.shape
#df.info()

,timestamp,action,reward,sensor_0_1_interaction,sensor_2_3_interaction,sensor_4_5_interaction,sensor_6_7_interaction,sensor_8_9_interaction,sensor_10_11_interaction,sensor_12_13_interaction,...,sensor_118_119_interaction,sensor_120_121_interaction,sensor_122_123_interaction,sensor_124_125_interaction,sensor_126_127_interaction,sensor_mean,sensor_std,sensor_diff_mean,sensor_roll_avg_mean,sensor_exp_mov_avg_mean
0,2024-01-31 19:43:28,6,0.144267,-0.067380,2.382772,-1.809039,0.341529,0.290694,0.819889,0.236015,...,-0.262242,0.919074,-0.049803,-0.566358,1.528715,-0.028778,1.024454,0.000000,0.000000,-0.028778
1,2024-01-31 19:43:29,9,-0.119299,0.037302,0.111864,0.682023,-1.887735,-0.084693,0.089305,0.502148,...,-0.244496,-2.261192,-0.219924,0.304074,-0.876549,0.107984,0.951637,0.136761,0.000000,0.053279
2,2024-01-31 19:43:30,3,0.289363,-0.818384,-0.519540,0.342660,-0.613077,-0.990719,1.449479,-1.828040,...,1.878976,-0.112477,0.371066,-0.675239,-0.241385,-0.104717,0.931194,-0.212700,0.000000,-0.021561
3,2024-01-31 19:43:31,1,-0.903657,1.062116,1.186834,-0.042364,0.275130,-0.354991,0.681410,-0.567441,...,-1.829413,0.180672,-1.975972,-0.570941,-0.374164,0.042963,1.050573,0.147680,0.000000,0.005241
4,2024-01-31 19:43:32,7,-0.644730,-0.120860,-0.732990,-0.443355,-2.002897,-0.219329,0.039804,-0.479130,...,0.014846,0.826288,-0.148734,-0.165662,-1.012201,0.026056,0.982468,-0.016907,0.008701,0.013232
5,2024-01-31 19:43:33,6,0.507407,0.006797,-0.459670,0.154752,2.419909,0.317906,-0.572989,-0.173676,...,0.731043,0.588162,-0.152102,0.964979,-0.947147,0.021796,0.982247,-0.004260,0.018816,0.016361
6,2024-01-31 19:43:34,4,-0.015535,0.363354,2.502763,-1.937506,-0.378011,1.610886,1.320598,0.024071,...,-0.007157,-1.397858,0.008454,2.223612,1.291235,-0.073518,1.030818,-0.095314,-0.017484,-0.015461
7,2024-01-31 19:43:35,9,-0.719344,-1.137152,-0.125549,-1.245519,0.522055,0.014942,-0.032810,0.817594,...,0.009535,-0.104391,0.037892,0.321399,-0.500313,0.028087,1.034647,0.101605,0.009077,-0.000356
8,2024-01-31 19:43:36,8,0.292694,-2.613457,0.054715,0.938682,0.134859,-0.711362,0.021579,-1.500971,...,-1.786859,0.108094,-0.081149,1.096611,1.078759,0.028526,1.012748,0.000439,0.006189,0.009529
9,2024-01-31 19:43:37,4,0.669401,0.719587,0.167825,-0.064800,2.578929,0.727049,-1.902966,0.062091,...,-0.200516,-1.317959,0.274647,0.026379,2.199135,0.064948,1.030950,0.036422,0.013968,0.028328



### Global Variables
Defining any constants and global variables used throughout the notebook.


In [8]:

# Adjust these parameters as needed for your model
seq_length = 128
d_model = 512
num_classes = 10



## Custom Layer Definitions

Here we define custom layers with appropriate documentation and naming conventions.



### BoolformerLayer

This custom TensorFlow layer performs a logical AND operation on its input and then processes it through a dense layer with ReLU activation.


In [9]:
class BoolformerLayer(layers.Layer):
    def __init__(self, threshold=0.5, **kwargs):
        super(BoolformerLayer, self).__init__(**kwargs)
        self.threshold = threshold

    def build(self, input_shape):
        self.dense_layer = layers.Dense(input_shape[-1], activation='relu')

    def call(self, inputs):
        boolean_inputs = tf.greater(inputs, self.threshold)  # Convert to boolean based on threshold
        logic_and = tf.math.logical_and(boolean_inputs, boolean_inputs)
        return self.dense_layer(tf.cast(logic_and, tf.float32))  # Convert back to float



### QLearningLayer

This layer is designed for reinforcement learning tasks, using a Q-learning algorithm to learn the quality of actions.


In [10]:
class QLearningLayer(layers.Layer):
    def __init__(self, action_space_size, learning_rate=0.01, gamma=0.95, **kwargs):
        super(QLearningLayer, self).__init__(**kwargs)
        self.action_space_size = action_space_size
        self.learning_rate = learning_rate
        self.gamma = gamma

    def build(self, input_shape):
        # A dense layer to process state and output Q-values for each action
        self.dense = layers.Dense(self.action_space_size, activation=None)

    def call(self, state, action=None, reward=None, next_state=None):
        q_values = self.dense(state)

        if action is not None and reward is not None and next_state is not None:
            # Get the predicted Q-values for the next state
            future_q_values = self.dense(next_state)
            max_future_q = tf.reduce_max(future_q_values, axis=1)

            # Compute the updated Q-value for the chosen action
            q_update = reward + self.gamma * max_future_q
            q_values_with_update = tf.tensor_scatter_nd_update(
                q_values, tf.expand_dims(action, axis=-1), q_update)

            # Update the Q-values
            self.dense.set_weights([q_values_with_update])

        return q_values


## Helper Functions

Defining helper functions such as positional encoding and transformer encoder with detailed comments for better understanding.



### Positional Encoding Function

Positional encoding adds information about the position of elements in the input sequence, crucial for models like transformers.


In [11]:
def positional_encoding(seq_length, d_model):
    position = tf.range(seq_length, dtype=tf.float32)[:, tf.newaxis]
    div_term = tf.exp(tf.range(0, d_model, 2, dtype=tf.float32) * -(tf.math.log(10000.0) / d_model))

    # Creating sine and cosine functions separately and then concatenating them
    sine_terms = tf.sin(position * div_term)
    cosine_terms = tf.cos(position * div_term)

    # Interleaving sine and cosine terms
    pos_encoding = tf.reshape(tf.concat([sine_terms, cosine_terms], axis=-1), [1, seq_length, d_model])

    return pos_encoding


### Transformer Encoder Function

The transformer encoder function applies transformations to the input data using layer normalization and multi-head attention, followed by a series of dense layers.


In [12]:

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Dense(ff_dim, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(inputs.shape[-1])(x)
    return x + res



## Model Building and Compilation

Here we build and compile the neural network model, ensuring clarity and efficiency in the code.



### Neural Network Model Creation Function

This function constructs the neural network using the previously defined custom layers and functions. It integrates the transformer encoder with the custom `BoolformerLayer` and `QLearningLayer`.


In [13]:
def create_neural_network_model():
    input_layer = keras.Input(shape=(seq_length, d_model))

    # Generate positional encoding and add it to the input
    pos_encoding = positional_encoding(seq_length, d_model)  # Ensure this returns a tensor
    pos_encoded = input_layer + pos_encoding

    # Transformer encoder
    transformer_output = transformer_encoder(inputs=pos_encoded, head_size=32, num_heads=2, ff_dim=64)

    # Custom layers (assuming these are correctly defined elsewhere)
    x_bool = BoolformerLayer()(transformer_output)
    rl_layer = QLearningLayer(action_space_size=num_classes)(x_bool)

    # Output layers
    output_layer = layers.Dense(num_classes, activation='softmax', name='Output')(rl_layer)
    reward_layer = layers.Dense(1, name='Reward')(rl_layer)

    # Constructing the model
    model = keras.Model(inputs=input_layer, outputs=[output_layer, reward_layer])

    # Compiling the model
    opt = optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
                  loss={'Output': 'categorical_crossentropy', 'Reward': 'mean_squared_error'},
                  metrics={'Output': 'accuracy'})

    return model

# Creating the model
model = create_neural_network_model()

# Displaying the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 512)]           0         []                            
                                                                                                  
 tf.__operators__.add (TFOp  (None, 128, 512)             0         ['input_1[0][0]']             
 Lambda)                                                                                          
                                                                                                  
 layer_normalization (Layer  (None, 128, 512)             1024      ['tf.__operators__.add[0][0]']
 Normalization)                                                                                   
                                                                                              


## Hyperparameter Settings

This section defines the hyperparameters for the model. Adjust these parameters to fine-tune the model's training process.


In [14]:

learning_rate = 0.001
batch_size = 32
epochs = 20
# Define additional hyperparameters here


## Model Training

In this section, we train the neural network model using the specified hyperparameters. The `model.fit()` function will be used to train the model with the training data. The validation data will be used to monitor the model's performance on unseen data.

In [ ]:
# Data Preparation
# Auto-detecting output and reward columns based on model architecture
output_label_column = df.columns[-2]  # Change this as per DataFrame Structure
reward_label_column = df.columns[-1]  # Change this as per DataFrame Structure
input_columns = df.drop([output_label_column, reward_label_column], axis=1)

# Standardize the input features
scaler = StandardScaler()
X = scaler.fit_transform(input_columns)

# Preparing output and reward labels
Y_output = df[output_label_column].values  # Assuming categorical labels
Y_reward = df[reward_label_column].values  # Assuming continuous values

# Splitting the dataset
X_train, X_val, Y_output_train, Y_output_val, Y_reward_train, Y_reward_val = train_test_split(
    X, Y_output, Y_reward, test_size=0.2, random_state=42
)

# Formatting data for model training
train_data = (X_train, {'Output': Y_output_train, 'Reward': Y_reward_train})
val_data = (X_val, {'Output': Y_output_val, 'Reward': Y_reward_val})

# Model Training
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1, mode='min'),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)
]

history = model.fit(
    train_data,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=val_data,
    verbose=1,
    callbacks=callbacks
)

# Model Saving
model.save('SephsRL.h5')

## Model Evaluation

After training the model, it's important to evaluate its performance on a test dataset to understand its efficacy. The following code will use the `model.evaluate()` function to assess the model's accuracy and loss on the test data.

In [ ]:
# Evaluate the model on the test data
# test_data = ...

evaluation_metrics = model.evaluate(test_data)
print(f"Test Loss: {evaluation_metrics[0]}, Test Accuracy: {evaluation_metrics[1]}")


## Visualizing Model Performance

Functions for plotting and analyzing the model's performance during training.


In [ ]:

import matplotlib.pyplot as plt

# Function to plot training history for both 'Output' and 'Reward' outputs, tailored to their characteristics
def plot_custom_output_history(history):
    num_plots = 2 + ('accuracy' in history.history)
    fig, axes = plt.subplots(1, num_plots, figsize=(6 * num_plots, 5))

    plot_index = 0

    # Plotting accuracy for 'Output', if it's available
    if 'accuracy' in history.history:
        axes[plot_index].plot(history.history['accuracy'], label='Training Accuracy - Output')
        axes[plot_index].plot(history.history['val_accuracy'], label='Validation Accuracy - Output')
        axes[plot_index].set_title('Accuracy for Output')
        axes[plot_index].set_xlabel('Epochs')
        axes[plot_index].set_ylabel('Accuracy')
        axes[plot_index].legend()
        plot_index += 1

    # Plotting loss for 'Output'
    axes[plot_index].plot(history.history['Output_loss'], label='Training Loss - Output')
    axes[plot_index].plot(history.history['val_Output_loss'], label='Validation Loss - Output')
    axes[plot_index].set_title('Loss for Output')
    axes[plot_index].set_xlabel('Epochs')
    axes[plot_index].set_ylabel('Loss')
    axes[plot_index].legend()
    plot_index += 1

    # Plotting loss for 'Reward'
    axes[plot_index].plot(history.history['Reward_loss'], label='Training Loss - Reward')
    axes[plot_index].plot(history.history['val_Reward_loss'], label='Validation Loss - Reward')
    axes[plot_index].set_title('Loss for Reward')
    axes[plot_index].set_xlabel('Epochs')
    axes[plot_index].set_ylabel('Loss')
    axes[plot_index].legend()

    plt.tight_layout()
    plt.show()

# Call the function with the training history
plot_custom_output_history(history)



## Conclusion

This notebook provided a detailed walkthrough for developing, training, and evaluating a neural network model with custom layers and advanced techniques, ensuring good coding practices and clear documentation throughout.
